### Caricamento librerie

In [28]:
#! pip install numpy
#! pip install pandas
#! pip install string

In [29]:
#Librerie generiche Python
import pandas as pd
import numpy as np
import pickle #Salvare Corpus 

#librerie per pre-processing
import gensim
import string
import multiprocessing #Numero di cpu
import re
from gensim.parsing.preprocessing import preprocess_string

#Librerie per i warnings
import warnings 
warnings.filterwarnings('ignore')

from tqdm import tqdm #Vedere l'avanzamento del run

### Caricamento Dataset per Topic Modeling

In [30]:
homepage = pd.read_csv("scrape_fin.csv", sep = ',', encoding='latin-1')

In [31]:
homepage

,Unnamed: 0,azienda,p.iva,provincia,regione,fatturato,sistema,dimensione,homepage + chi siamo
0,48,BOFFI SPA (MB),2642250969,Monza e della Brianza,Lombardia,49277513,Sistema ARREDAMENTO,media,Urban living spaces Kitchenology Italian craft...
1,55,GIORGETTI SPA,695300962,Monza e della Brianza,Lombardia,43667553,Sistema ARREDAMENTO,media,OBJECT TO PROJECT The travelling exhibition ce...
2,67,BAXTER SRL,913080966,Monza e della Brianza,Lombardia,37981607,Sistema ARREDAMENTO,media,ABOUT MOOD LEATHER MATERIAL PRODUCT DESIGNER P...
3,96,CAP DESIGN SPA,4311900965,Monza e della Brianza,Lombardia,27935275,Sistema ARREDAMENTO,media,opened a new shop in shop in Beijing with a se...
4,102,FLOU SPA,779080969,Monza e della Brianza,Lombardia,25552326,Sistema ARREDAMENTO,media,Back in 1978 Vico Magistretti designed Nathali...
5,130,MERIDIANI SRL,6926770964,Monza e della Brianza,Lombardia,19861276,Sistema ARREDAMENTO,media,IMM COLOGNE MAISON&OBJET 01-2019 Meridiani con...
6,143,LIVING DIVANI SRL,786120964,Monza e della Brianza,Lombardia,18220000,Sistema ARREDAMENTO,media,"Company ""MERIDIANI was born in 1996 from a si..."
7,158,ZANOTTA SPA,695980961,Monza e della Brianza,Lombardia,16106441,Sistema ARREDAMENTO,media,"Established in 1954, Zanotta is one of the rec..."
8,178,LONGHI SPA,687180968,Monza e della Brianza,Lombardia,14721275,Sistema ARREDAMENTO,media,"Established in 1954, Zanotta is one of the rec..."
9,199,CAIMI BREVETTI SPA,889250965,Monza e della Brianza,Lombardia,12992669,Sistema ARREDAMENTO,media,"Established in 1949, Caimi Brevetti S.p.A. is ..."


### Text Pre-processing 

In [32]:
# Rimozione NaN
# Da 26 aziende a 24
homepage = homepage.dropna(subset=['homepage + chi siamo'])
homepage

,Unnamed: 0,azienda,p.iva,provincia,regione,fatturato,sistema,dimensione,homepage + chi siamo
0,48,BOFFI SPA (MB),2642250969,Monza e della Brianza,Lombardia,49277513,Sistema ARREDAMENTO,media,Urban living spaces Kitchenology Italian craft...
1,55,GIORGETTI SPA,695300962,Monza e della Brianza,Lombardia,43667553,Sistema ARREDAMENTO,media,OBJECT TO PROJECT The travelling exhibition ce...
2,67,BAXTER SRL,913080966,Monza e della Brianza,Lombardia,37981607,Sistema ARREDAMENTO,media,ABOUT MOOD LEATHER MATERIAL PRODUCT DESIGNER P...
3,96,CAP DESIGN SPA,4311900965,Monza e della Brianza,Lombardia,27935275,Sistema ARREDAMENTO,media,opened a new shop in shop in Beijing with a se...
4,102,FLOU SPA,779080969,Monza e della Brianza,Lombardia,25552326,Sistema ARREDAMENTO,media,Back in 1978 Vico Magistretti designed Nathali...
5,130,MERIDIANI SRL,6926770964,Monza e della Brianza,Lombardia,19861276,Sistema ARREDAMENTO,media,IMM COLOGNE MAISON&OBJET 01-2019 Meridiani con...
6,143,LIVING DIVANI SRL,786120964,Monza e della Brianza,Lombardia,18220000,Sistema ARREDAMENTO,media,"Company ""MERIDIANI was born in 1996 from a si..."
7,158,ZANOTTA SPA,695980961,Monza e della Brianza,Lombardia,16106441,Sistema ARREDAMENTO,media,"Established in 1954, Zanotta is one of the rec..."
8,178,LONGHI SPA,687180968,Monza e della Brianza,Lombardia,14721275,Sistema ARREDAMENTO,media,"Established in 1954, Zanotta is one of the rec..."
9,199,CAIMI BREVETTI SPA,889250965,Monza e della Brianza,Lombardia,12992669,Sistema ARREDAMENTO,media,"Established in 1949, Caimi Brevetti S.p.A. is ..."


In [33]:
#Funzione di Text Cleaning
def text_preproc(text:str, log:bool=True):

    my_filter = [
    lambda x: x.lower(),
    gensim.parsing.strip_punctuation,
    gensim.parsing.strip_multiple_whitespaces,
    gensim.parsing.strip_numeric,
    gensim.parsing.remove_stopwords,  
    gensim.parsing.strip_tags,
    gensim.parsing.stem_text,

    ]
    
    return preprocess_string(text, filters=my_filter)

In [34]:
#Applicazione funzione al testo
homepage['text_clean'] = homepage['homepage + chi siamo'].apply(text_preproc)

In [35]:
#Rename column
homepage = homepage.rename(index=str, columns={"homepage + chi siamo": "homepage"})

### Topic Modeling

### Creazione Dizionari con Keywords

In [36]:
smart_keys={ 'innov':'innov', 'technolog':'technolog',
            'smart':'smart',
            'digit':'digit','iot':'iot','research':'research','develop':'develop', 
            'autom':'autom', 'sensor':'sensor', 'sensi':'sensori',
             'avant-grad':'avant-grad',  'intellig':'intellig', 
            'insight':'insight', 'devic':'devic', 'interfac':
           'interfac','revolut':'revolut', 'future':'future', 'flexib': 'flexib',
           'kitchenolog': 'kitchenolog'}

design_keys={'design': 'design', 'style':'style',
             'stylish':'stylish','comfort':'comfort','fashion':'fashion',
             'architectur':'architectur', 'opera':'opera', 'creativ':'creativ','opera':'opera'}

env_keys={ 'eco':'eco', 'ecolog':'ecolog', 'sustain':'sustain','green':'green',
          'environ':'environ', 'resourc':'resourc', 'natur':'natur','pollut':'pollut',
            'recycl':'recycl', 'natur':'natur','fresh':'fresh','organ':'organ','bio':'bio', 'biodegrad':
          'biodegrad','compost':'complost','biodivers':'biodivers',
          'effici':'effici','wast':'wast', 'renewv':'renewv','impact':'impact', 'faux': 'faux'}


social_net_keys={'share':'share','shar':'shar','network':'network','networks':'networks','linkedin':'linkedin',
                      'instagram':'instagram','facebook':'facebook','social':'social','media':'media', 'follow': 'follow'}

### Check Keywords in Text

In [37]:
# Prima funzione: se c'è almeno una keyword nella homepage = true altrimenti false.
# Seconda funzione: se la keyword è presente nella homepage scrive la parola.

def smart(input_str):
    for i in smart_keys.values():
        if i in input_str:
            return True
        return False      
        
homepage.loc[:, 'smart'] = homepage.text_clean.apply(smart)

def smart_in_text(text):
    if(smart(text)):
        parole_trovate = list()
        for i in smart_keys.values():
            if i in text:
                parole_trovate.append(i)
        return parole_trovate 

homepage['smart_words'] = homepage.text_clean.apply(smart_in_text)

#-------------------------------------------------------------------------------------------------

def design(input_str):
    for i in design_keys.values():
        if i in input_str:
            return True
        return False      
        
homepage.loc[:, 'design'] = homepage.text_clean.apply(design)

def design_in_text(text):
    if(design(text)):
        parole_trovate = list()
        for i in design_keys.values():
            if i in text:
                parole_trovate.append(i)
        return parole_trovate 

homepage['design_words'] = homepage.text_clean.apply(design_in_text)

#-----------------------------------------------------------------------------------------------

def env(input_str):
    for i in env_keys.values():
        if i in input_str:
            return True
        return False      
        
homepage.loc[:, 'env'] = homepage.text_clean.apply(env)

def env_in_text(text):
    if(env(text)):
        parole_trovate = list()
        for i in env_keys.values():
            if i in text:
                parole_trovate.append(i)
        return parole_trovate 

homepage['env_words'] = homepage.text_clean.apply(env_in_text)

#-----------------------------------------------------------------------------------------------

def soc_net(input_str):
    for i in social_net_keys.values():
        if i in input_str:
            return True
        return False      
        
homepage.loc[:, 'soc_net'] = homepage.text_clean.apply(soc_net)

def soc_net_in_text(text):
    if(soc_net(text)):
        parole_trovate = list()
        for i in social_net_keys.values():
            if i in text:
                parole_trovate.append(i)
        return parole_trovate 

homepage['soc_net_words'] = homepage.text_clean.apply(soc_net_in_text)

In [38]:
homepage

,Unnamed: 0,azienda,p.iva,provincia,regione,fatturato,sistema,dimensione,homepage,text_clean,smart,smart_words,design,design_words,env,env_words,soc_net,soc_net_words
0,48,BOFFI SPA (MB),2642250969,Monza e della Brianza,Lombardia,49277513,Sistema ARREDAMENTO,media,Urban living spaces Kitchenology Italian craft...,"[urban, live, space, kitchenolog, italian, cra...",True,"[innov, technolog, develop, kitchenolog]",True,"[design, creativ]",False,None,True,"[share, network, instagram, social]"
1,55,GIORGETTI SPA,695300962,Monza e della Brianza,Lombardia,43667553,Sistema ARREDAMENTO,media,OBJECT TO PROJECT The travelling exhibition ce...,"[object, project, travel, exhibit, celebr, th,...",True,"[innov, technolog, digit]",True,"[design, style, creativ]",True,"[eco, natur, effici]",False,None
2,67,BAXTER SRL,913080966,Monza e della Brianza,Lombardia,37981607,Sistema ARREDAMENTO,media,ABOUT MOOD LEATHER MATERIAL PRODUCT DESIGNER P...,"[mood, leather, materi, product, design, press...",False,None,True,[design],False,None,False,None
3,96,CAP DESIGN SPA,4311900965,Monza e della Brianza,Lombardia,27935275,Sistema ARREDAMENTO,media,opened a new shop in shop in Beijing with a se...,"[open, new, shop, shop, beij, select, import, ...",True,"[innov, research]",True,"[design, architectur]",False,None,False,None
4,102,FLOU SPA,779080969,Monza e della Brianza,Lombardia,25552326,Sistema ARREDAMENTO,media,Back in 1978 Vico Magistretti designed Nathali...,"[vico, magistretti, design, nathali, origin, m...",True,"[innov, technolog, develop, revolut]",True,"[design, style, stylish, comfort]",False,None,False,None
5,130,MERIDIANI SRL,6926770964,Monza e della Brianza,Lombardia,19861276,Sistema ARREDAMENTO,media,IMM COLOGNE MAISON&OBJET 01-2019 Meridiani con...,"[imm, cologn, maison, objet, meridiani, consol...",False,None,False,None,False,None,False,None
6,143,LIVING DIVANI SRL,786120964,Monza e della Brianza,Lombardia,18220000,Sistema ARREDAMENTO,media,"Company ""MERIDIANI was born in 1996 from a si...","[compani, meridiani, born, simpl, passion, ide...",True,"[innov, digit, research, develop]",True,"[design, style, creativ]",False,None,True,[share]
7,158,ZANOTTA SPA,695980961,Monza e della Brianza,Lombardia,16106441,Sistema ARREDAMENTO,media,"Established in 1954, Zanotta is one of the rec...","[establish, zanotta, recogn, leader, italian, ...",True,"[innov, technolog, research, insight]",True,"[design, style]",False,None,False,None
8,178,LONGHI SPA,687180968,Monza e della Brianza,Lombardia,14721275,Sistema ARREDAMENTO,media,"Established in 1954, Zanotta is one of the rec...","[establish, zanotta, recogn, leader, italian, ...",True,"[innov, technolog, research, insight]",True,"[design, style, creativ]",False,None,False,None
9,199,CAIMI BREVETTI SPA,889250965,Monza e della Brianza,Lombardia,12992669,Sistema ARREDAMENTO,media,"Established in 1949, Caimi Brevetti S.p.A. is ...","[establish, caimi, brevetti, s, p, lead, manuf...",True,"[innov, technolog, research, develop]",True,"[design, comfort]",False,None,False,None


### Count parole

In [39]:
smart = []
for i, el in enumerate(homepage['smart_words']):
    if el != None:
        l = []
        for item in el:
            a = homepage['text_clean'][i].count(item)
            l.append(a)
        smart.append(l)
    else:
        smart.append(0)

In [40]:
design = []
for i, el in enumerate(homepage['design_words']):
    if el != None:
        l = []
        for item in el:
            a = homepage['text_clean'][i].count(item)
            l.append(a)
        design.append(l)
    else:
        design.append(0)

In [41]:
sust = []
for i, el in enumerate(homepage['env_words']):
    if el != None:
        l = []
        for item in el:
            a = homepage['text_clean'][i].count(item)
            l.append(a)
        sust.append(l)
    else:
        sust.append(0)

In [42]:
socnet = []
for i, el in enumerate(homepage['soc_net_words']):
    if el != None:
        l = []
        for item in el:
            a = homepage['text_clean'][i].count(item)
            l.append(a)
        socnet.append(l)
    else:
        socnet.append(0)

In [43]:
homepage['smart_count'] = smart
homepage['design_count'] = design
homepage['sust_count'] = sust
homepage['soc_net_count'] = socnet

### Sum del count

In [44]:
tab_finale_parsec = homepage[['azienda', 'sistema', 'dimensione', 'smart_count', 'design_count', 'sust_count', 'soc_net_count']]
tab_finale_parsec

,azienda,sistema,dimensione,smart_count,design_count,sust_count,soc_net_count
0,BOFFI SPA (MB),Sistema ARREDAMENTO,media,"[1, 1, 1, 1]","[7, 1]",0,"[2, 1, 1, 1]"
1,GIORGETTI SPA,Sistema ARREDAMENTO,media,"[7, 7, 1]","[5, 7, 1]","[2, 1, 2]",0
2,BAXTER SRL,Sistema ARREDAMENTO,media,0,[2],0,0
3,CAP DESIGN SPA,Sistema ARREDAMENTO,media,"[1, 1]","[8, 1]",0,0
4,FLOU SPA,Sistema ARREDAMENTO,media,"[7, 1, 4, 1]","[10, 1, 1, 2]",0,0
5,MERIDIANI SRL,Sistema ARREDAMENTO,media,0,0,0,0
6,LIVING DIVANI SRL,Sistema ARREDAMENTO,media,"[1, 1, 1, 1]","[9, 2, 1]",0,[1]
7,ZANOTTA SPA,Sistema ARREDAMENTO,media,"[1, 3, 1, 1]","[10, 1]",0,0
8,LONGHI SPA,Sistema ARREDAMENTO,media,"[1, 4, 2, 1]","[14, 1, 1]",0,0
9,CAIMI BREVETTI SPA,Sistema ARREDAMENTO,media,"[2, 7, 3, 1]","[9, 1]",0,0


In [45]:
tab_finale_parsec['smart_count'] = tab_finale_parsec['smart_count'].apply(lambda x : np.sum(x))
tab_finale_parsec['design_count'] = tab_finale_parsec['design_count'].apply(lambda x : np.sum(x))
tab_finale_parsec['sust_count'] = tab_finale_parsec['sust_count'].apply(lambda x : np.sum(x))
tab_finale_parsec['soc_net_count'] = tab_finale_parsec['soc_net_count'].apply(lambda x : np.sum(x))

In [48]:
tab_finale_parsec

,azienda,sistema,dimensione,smart_count,design_count,sust_count,soc_net_count
0,BOFFI SPA (MB),Sistema ARREDAMENTO,media,4,8,0,5
1,GIORGETTI SPA,Sistema ARREDAMENTO,media,15,13,5,0
2,BAXTER SRL,Sistema ARREDAMENTO,media,0,2,0,0
3,CAP DESIGN SPA,Sistema ARREDAMENTO,media,2,9,0,0
4,FLOU SPA,Sistema ARREDAMENTO,media,13,14,0,0
5,MERIDIANI SRL,Sistema ARREDAMENTO,media,0,0,0,0
6,LIVING DIVANI SRL,Sistema ARREDAMENTO,media,4,12,0,1
7,ZANOTTA SPA,Sistema ARREDAMENTO,media,6,11,0,0
8,LONGHI SPA,Sistema ARREDAMENTO,media,8,16,0,0
9,CAIMI BREVETTI SPA,Sistema ARREDAMENTO,media,13,10,0,0
